In [2]:
!pip install transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.0 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 895 kB 71.5 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")



Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6G [00:00<?, ?B/s]

<pad> Positive</s>


In [16]:
import pandas as pd

df =  pd.read_excel('/content/semantic_baseline_records_all_V1.xlsx')
df

,Q,Question,Company Name,url,html_text,found_noun,context,context_verb,confidence,Status,New Context,Avg Sim,Comments,id,answer,score,Unnamed: 6
0,Q1,"Does the insured/applicant process, transmit, ...",DC ASSOCIATES,http://procaremd.com/?page_id=15,Like us on Facebook . PATIENT PORTAL LOGIN . O...,['health information'],a referral . Patient Forms . Patient Registrat...,['record'],0.99,TP,Welcome to ProCare Internal Medicine Associate...,0.434092,there is a discrepency with the URL and the co...,NaN,NaN,NaN,NaN
1,Q1,"Does the insured/applicant process, transmit, ...",HEALTH EXCEL INCORPORATED,https://greatnonprofits.org/org/health-excel-f...,Donate . Share . GreatNonprofits . Over millio...,['health information'],education the dissemination of health care inf...,['use'],0.99,FP,To improve the health and delivery of health c...,0.330500,NaN,NaN,NaN,NaN,NaN
2,Q1,"Does the insured/applicant process, transmit, ...",BAYLOR MEDICAL FOUNDATION,https://www.bcm.edu/healthcare/specialties/fam...,COVID Response . Access our . COVID Response h...,['health information'],Medical Home Recognition raises the bar in def...,['use'],0.99,FP,Primary Care Sports Medicine focuses on health...,0.399539,NaN,NaN,NaN,NaN,NaN
3,Q1,"Does the insured/applicant process, transmit, ...",BENCHMARK DENTAL PLANS,https://www.dentalplans.com/dentists/tx/austin...,My Cart . Member Login . En Espaol . Join Now ...,['health data'],Without Plan] is determined by DentalPlanscom ...,['use'],0.99,TP,Provide affordable solutions to uninsured pati...,0.378895,NaN,NaN,NaN,NaN,NaN
4,Q1,"Does the insured/applicant process, transmit, ...",INDUSTRIAL PROCESS COMPONENTS INCORPORATED,https://www.phoenix.edu/campus-locations/tx/fo...,Skip to Bottom . Skip to Chat Email Text . Sit...,['health information'],. Marketing . Operations Management . Project ...,['record'],0.99,TP,Bachelor of Science in Health Administration ....,0.510338,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Q8,Does the insured/applicant perform any service...,NaN,https://www.amsoil.com/,NaN,NaN,The AMSOIL Preferred Customer Program gives yo...,NaN,NaN,FP,Stores Maintenance info for all your vehicles ...,0.343948,NaN,NaN,prohibited,0.005323,NaN
231,Q8,Does the insured/applicant perform any service...,NaN,https://www.jacksonhewitt.com/tax-preparation-...,NaN,NaN,Find an office . Wheres My Refund . Refund Adv...,NaN,NaN,TP,With the Worryfree Guarantee Platinum if the I...,0.389195,NaN,NaN,no Exclusion defined below applies,0.000179,NaN
232,Q8,Does the insured/applicant perform any service...,NaN,https://healthprovidersdata.com/hipaa/codes/NP...,NaN,NaN,Home . NPI Lookup . HCPCS Lookup . ICD Lookup ...,NaN,NaN,TP,The NPI record includes the healthcare provide...,0.393974,NaN,NaN,National Provider Identifier,0.002247,NaN
233,Q8,Does the insured/applicant perform any service...,NaN,https://healthprovidersdata.com/hipaa/codes/NP...,NaN,NaN,Home . NPI Lookup . HCPCS Lookup . ICD Lookup ...,NaN,NaN,TP,The NPI record includes the healthcare provide...,0.391560,NaN,NaN,National Provider Identifier,0.003996,NaN


In [30]:

def t5_prompt_design():
    prompt_stmt = ""
    for index,row in df[:4].iterrows():
        hypothesis = row['Question']
        premise = row['New Context']
        if row['Status'] == 'TP': 
            true_label = True
        if row['Status'] == 'FP':
            true_label = False
        else:
            true_label = False

        stmt  = "{} Question: {} True or False? Answer: {} ".format(premise,hypothesis,true_label)
        print(stmt)
        
        prompt_stmt += stmt
    return prompt_stmt

prompt_stmt = t5_prompt_design()

##initialize T0 model with the prompt input
inputs = tokenizer.encode(prompt_stmt, return_tensors="pt")

print('Grenerate Module ')

for index,row in df[:10].iterrows():
    hypothesis = row['Question']
    premise = row['New Context']
    stmt  = "{} Question: {} True or False? Answer: ".format(premise,hypothesis)
    print(stmt)
    true_label = row['Status']
    print(true_label)
    inputs = tokenizer.encode(stmt, return_tensors="pt")
    outputs = model.generate(inputs)
    print(tokenizer.decode(outputs[0]))
    print('\n\n')


Welcome to ProCare Internal Medicine Associates patient portal We are excited to bring you this technology to communicate and access your health care needs . With the portal you will be able to request appointments prescription refills and referrals In addition you will have access to your medical record to include lab results visit summaries and be able to communicate with our staff . Welcome to ProCare Our patient forms are available on this website for your completionreview if you would like to complete before you arrive otherwise we ask that you arrive minutes before your scheduled appointment . Please bring your insurance card and a picture ID to your visit . Thank you for choosing ProCare Internal Medicine Associates as your home for all your health care needs We look forward to having you in our health care family . Question: Does the insured/applicant process, transmit, collect, record, store or otherwise handle non-employee Protected Health Information (PHI) or other health/fi

In [13]:
premise = 'GROUP INC is located in Brooklyn NY and is . happy to offer our expertise in all facets of  Security Consulting  Training Our tried and true system is based off over years cumulative experience shared between'
hypothesis = 'Does the insured/applicant provide B2C information security software, hardware or services? Or, do they offer information security consulting on either a B2B or B2C basis?'
stmt  = "Given {} Should we assume that {} is true? Yes or no?".format(premise,hypothesis)
print(stmt)

inputs = tokenizer.encode(stmt, return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

Given GROUP INC is located in Brooklyn NY and is . happy to offer our expertise in all facets of  Security Consulting  Training Our tried and true system is based off over years cumulative experience shared between Should we assume that Does the insured/applicant provide B2C information security software, hardware or services? Or, do they offer information security consulting on either a B2B or B2C basis? is true? Yes or no?
<pad> No</s>


In [8]:
print(tokenizer.decode(outputs[0]))

<pad> Positive</s>
